In [1]:
import pandas as pd
import numpy as np
from math import log2

In [2]:
# Functions

def Gini(Ls, WW = None):
    
    
    if WW == None:
        L = len(Ls)
        WW = [1]*L
    
    W = {}
    Sum = 0
    for x in range(len(Ls)):
        
        if Ls[x] not in W:
            W[Ls[x]] = 0
            
        W[Ls[x]] += WW[x]
        Sum += WW[x]
    
    S = 0
    for x in W:
        S += (W[x]/Sum)**2
        
    return 1 - S

#################################################################


def majority(Ls, WW = None):
    
    
  
    if WW == None:
        L = len(Ls)
        WW = [1]*L
    
    W = {}
    for x in range(len(Ls)):
        
        if Ls[x] not in W:
            W[Ls[x]] = 0
            
        W[Ls[x]] += WW[x]
    
    Max = -1
    majority = None    
    for y in W:
        if W[y] > Max:
            Max = W[y]
            majority = y        
    return 1 - Max/sum(WW)

#################################################################

def entropy(Ls, WW = None):
    
    n = len(Ls)
    if WW == None:
        WW = [1]*n
            
    W = {}
    Sum = 0
    for x in range(n):
        if Ls[x] not in W:
            W[Ls[x]] = 0
        
        W[Ls[x]] += WW[x]
            
        Sum += WW[x]
        
    S = 0
    for x in W:
        S += (W[x]/Sum) * log2(Sum / W[x])

    return S


#################################################################

def Majority(Ls, WW = None, ignor = None):
    
    
    if WW == None:
        L = len(Ls)
        WW = [1]*L
    
    W = {}
    for x in range(len(Ls)):
        
        if Ls[x] not in W:
            W[Ls[x]] = 0
        
        if ignor is None:
            W[Ls[x]] += WW[x]
        else:
            if Ls[x]!= ignor:
                W[Ls[x]] += WW[x]
                
    
    Max = -1
    majority = None    
    for y in W:
        if W[y] > Max:
            Max = W[y]
            majority = y        
    return(majority, len(W))



#################################################################


def Entropy_given_attribute(T, L, a, W = None, Entropy_function = entropy):
    
    n = len(L)
    if W == None:
        W = [1]*n
    
    
    split_l = {}
    split_w = {}
    sum_W = sum(W)
    
    for x in range(n):
        
        u = T[x][a]
        if u not in split_w:
            
            split_w[u] =[]
            split_l[u] = []
            
        split_w[u].append(W[x])
        split_l[u].append(L[x])  
        
    En = 0        
    for x in split_w:
        
        En += sum(split_w[x]) * Entropy_function(split_l[x], split_w[x]) / sum_W
        
    return(En, list(split_w.keys()))    


#################################################################


class DT(object):
    def __init__(self, train, L, attss, depth = -1, WW = None, 
                 Entropy_function = entropy):
        
        self.Entropy_function = Entropy_function
        self.leaf = False 
        self.label, n_values = Majority(L, WW) 
        
        if len(attss) == 0 or n_values == 1 or depth == 0:
            
            self.leaf = True  
            return
        
        self.at_splt, values = self.best_att(train, L, attss, WW)
        
        train_s, lables_s, weight_s = split(train, L, self.at_splt, WW) 
        self.Tree = {}
        
        attss.remove(self.at_splt)
        
        for v in train_s:
               
            self.Tree[v] = DT(train_s[v], lables_s[v], attss, depth - 1, 
                                        weight_s[v], Entropy_function)

        attss.append(self.at_splt)
            
    
    def predict(self, instance):
        
        if self.leaf:
            return self.label
        
        if instance[self.at_splt] in self.Tree:
            return self.Tree[instance[self.at_splt]].predict(instance)   
        
        return self.label   
    
    
    def best_att(self, train, L, attss, WW):
        
        lable_Ent = entropy(L, WW)
        Max = -1
        Best = None
        Best_values = None
    
        for attribute in attss: 
            temp, temp_values = Entropy_given_attribute(train, L, 
                                                        attribute, WW, self.Entropy_function) 
            if lable_Ent - temp >  Max:
                Max = lable_Ent - temp
                Best = attribute
                Best_values = temp_values
                    
        return(Best, Best_values)

#################################################################    
    
def split(train, label, attribute, WW = None):
    
    n = len(label)
    if WW == None:
        WW = [1]*n
    
    split_w = {}
    split_t = {}
    split_l = {}
    
    for x in range(len(label)):
        
        #print('x = ', x)
        #print('attribute = ', attribute)
        txa = train[x][attribute]
        if txa not in split_t:
            
            split_w[txa] = []
            split_t[txa] = []
            split_l[txa] = []
            
        split_w[txa].append(WW[x])
        split_t[txa].append(train[x])
        split_l[txa].append(label[x])
        
    return (split_t, split_l, split_w)

In [3]:
attributesfile = open('car/data-desc.txt', 'r')
print(attributesfile.read())

| label values

unacc, acc, good, vgood

| attributes

buying:   vhigh, high, med, low.
maint:    vhigh, high, med, low.
doors:    2, 3, 4, 5more.
persons:  2, 4, more.
lug_boot: small, med, big.
safety:   low, med, high.

| columns
buying,maint,doors,persons,lug_boot,safety,label



In [4]:
C = [0, 1, 2, 3, 4, 5, 6]

In [5]:
train = pd.read_csv('car/train.csv', names = C)
test = pd.read_csv('car/test.csv', names = C)
train.head()

,0,1,2,3,4,5,6
0,low,vhigh,4,4,big,med,acc
1,low,high,5more,4,med,high,vgood
2,vhigh,med,2,2,big,high,unacc
3,high,high,2,2,small,high,unacc
4,vhigh,low,3,2,big,low,unacc


In [6]:
#list(train.loc[0])

In [7]:
Train = []
Label = []
for i in range(len(train)):
    temp = list(train.loc[i])
    Train.append(temp[:-1])
    Label.append(temp[-1])

In [8]:
Test = []
Test_Label = []
for i in range(len(test)):
    temp = list(train.loc[i])
    Test.append(temp[:-1])
    Test_Label.append(temp[-1])

In [9]:
attributes = C.copy()
attributes.remove(6)

In [10]:
attributes

[0, 1, 2, 3, 4, 5]

In [11]:
# import time
# start =time.time()
# A = DecisionTree(Train, Label, attributes, depth = 10, Entropy_function = Gini)
# duration = time.time() - start
# print('duration {}'.format(duration))

In [12]:
def prediction(Model, data, label):
    pred = []
    for i in range(len(data)):
        temp = Model.predict(data[i])
        pred.append(temp)
    return (np.array(label) == np.array(pred)).mean()

In [13]:
for f in [entropy, majority, Gini]:
    print("the method is baces on ", f, "index")
    for depth in range(1,7):
        Model = DT(Train, Label, attributes, depth = depth, Entropy_function = f)
        print("for depth = {}, error for training is {}".format(depth, 1-prediction(Model, Train, Label)))

the method is baces on  <function entropy at 0x7fa241840430> index
for depth = 1, error for training is 0.30200000000000005
for depth = 2, error for training is 0.22199999999999998
for depth = 3, error for training is 0.18100000000000005
for depth = 4, error for training is 0.08199999999999996
for depth = 5, error for training is 0.027000000000000024
for depth = 6, error for training is 0.0
the method is baces on  <function majority at 0x7fa241840280> index
for depth = 1, error for training is 0.30200000000000005
for depth = 2, error for training is 0.29800000000000004
for depth = 3, error for training is 0.17400000000000004
for depth = 4, error for training is 0.18000000000000005
for depth = 5, error for training is 0.052000000000000046
for depth = 6, error for training is 0.0
the method is baces on  <function Gini at 0x7fa2418401f0> index
for depth = 1, error for training is 0.30200000000000005
for depth = 2, error for training is 0.22199999999999998
for depth = 3, error for training

In [14]:
for f in [entropy, majority, Gini]:
    print("the method is baces on ", f, "index")
    for depth in range(1,7):
        Model = DT(Train, Label, attributes, depth = depth, Entropy_function = f)
        print("for depth = {}, error for test is {}".format(depth, 1-prediction(Model, Test, Test_Label)))

the method is baces on  <function entropy at 0x7fa241840430> index
for depth = 1, error for test is 0.30082417582417587
for depth = 2, error for test is 0.22664835164835162
for depth = 3, error for test is 0.17582417582417587
for depth = 4, error for test is 0.07829670329670335
for depth = 5, error for test is 0.02472527472527475
for depth = 6, error for test is 0.0
the method is baces on  <function majority at 0x7fa241840280> index
for depth = 1, error for test is 0.30082417582417587
for depth = 2, error for test is 0.2953296703296703
for depth = 3, error for test is 0.17307692307692313
for depth = 4, error for test is 0.17307692307692313
for depth = 5, error for test is 0.04670329670329665
for depth = 6, error for test is 0.0
the method is baces on  <function Gini at 0x7fa2418401f0> index
for depth = 1, error for test is 0.30082417582417587
for depth = 2, error for test is 0.22664835164835162
for depth = 3, error for test is 0.17307692307692313
for depth = 4, error for test is 0.0892